In [1]:
%%time
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import math
import xgboost as xgb
from scipy.stats import skew
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, auc, classification_report, roc_curve, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime
# You'll use en_core_web_sm for the sprint challenge due memory constraints on Codegrader
#!python -m spacy download en_core_web_sm 

# Locally (or on colab) let's use en_core_web_lg 
#!python -m spacy download en_core_web_md 


CPU times: user 664 ms, sys: 160 ms, total: 823 ms
Wall time: 612 ms


In [2]:

# import en_core_web_md

In [3]:
import re
import string

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# import spacy

# from bs4 import BeautifulSoup
%matplotlib inline


# nlp = spacy.load("en_core_web_md")

In [4]:
import pickle as pkl


In [5]:
train = pd.read_csv('/Users/colerichards/Downloads/archive/train.csv')
test = pd.read_csv('/Users/colerichards/Downloads/archive/test.csv')

In [6]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,18,1
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,1,0


In [7]:
train.nunique()

project_id               108129
name                     107838
desc                     107377
goal                       3567
keywords                 108129
disable_communication         2
country                      11
currency                      9
deadline                 100933
state_changed_at         102417
created_at               108062
launched_at              108039
backers_count              2426
final_status                  2
dtype: int64

In [8]:
train.isnull().sum()

project_id               0
name                     3
desc                     9
goal                     0
keywords                 0
disable_communication    0
country                  0
currency                 0
deadline                 0
state_changed_at         0
created_at               0
launched_at              0
backers_count            0
final_status             0
dtype: int64

In [9]:
train['log_goal'] = np.log10(train['goal'])

In [10]:
train.head(5)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status,log_goal
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1,1.301030
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0,2.477121
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0,1.477121
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,18,1,2.698970
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,1,0,3.301030


In [11]:
train.loc[train['log_goal'] <= 1  , 'log_goal'] = 0 
train.loc[(train['log_goal'] >1 ) & (train['log_goal']<= 3) , 'log_goal'] = 1  
train.loc[train['log_goal'] > 3 & (train['log_goal']<=5), 'log_goal'] = 2  
train.loc[train['log_goal'] > 5 , 'log_goal'] = 3

In [12]:
train.head(5)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status,log_goal
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1,1.0
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0,1.0
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,1243027560,1243027818,1242163613,1242164398,0,0,1.0
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,1243555740,1243556121,1240963795,1240966730,18,1,1.0
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,1243769880,1243770317,1241177914,1241180541,1,0,2.0


In [13]:
date_time = ['deadline', 'state_changed_at', 'created_at', 'launched_at']
for i in date_time:
    train[i] = train[i].apply(lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

In [14]:
for i in date_time:
    test[i] = test[i].apply(lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S'))

In [15]:
def Duration(d):
    launch = datetime.strptime(d[0], '%Y-%m-%d %H:%M:%S')
    deadline = datetime.strptime(d[1], '%Y-%m-%d %H:%M:%S')
    duration = deadline-launch
    weekDifference = int(duration.total_seconds()/(3600*24*7))
    return weekDifference
train['duration'] = train[['launched_at', 'deadline']].apply(lambda d: Duration(d), axis=1)

In [16]:
test['duration'] = test[['launched_at', 'deadline']].apply(lambda d: Duration(d), axis=1)

In [17]:
train.head()

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status,log_goal,duration
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,2009-05-02 23:59:59,2009-05-03 00:00:17,2009-04-24 12:15:07,2009-04-24 12:52:03,3,1,1.0,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,2009-05-15 16:10:00,2009-05-15 17:00:18,2009-04-28 16:10:24,2009-04-28 20:26:32,2,0,1.0,2
2,kkst183622197,Mr. Squiggles,So I saw darkpony's successfully funded drawin...,30.0,mr-squiggles,False,US,USD,2009-05-22 14:26:00,2009-05-22 14:30:18,2009-05-12 14:26:53,2009-05-12 14:39:58,0,0,1.0,1
3,kkst597742710,Help me write my second novel.,Do your part to help out starving artists and ...,500.0,help-me-write-my-second-novel,False,US,USD,2009-05-28 17:09:00,2009-05-28 17:15:21,2009-04-28 17:09:55,2009-04-28 17:58:50,18,1,1.0,4
4,kkst1913131122,Support casting my sculpture in bronze,"I'm nearing completion on a sculpture, current...",2000.0,support-casting-my-sculpture-in-bronze,False,US,USD,2009-05-31 04:38:00,2009-05-31 04:45:17,2009-05-01 04:38:34,2009-05-01 05:22:21,1,0,2.0,4


In [18]:
train['final_status'].value_counts()

0    73568
1    34561
Name: final_status, dtype: int64

In [19]:
test['log_goal'] = np.log10(test['goal'])

In [20]:
test.insert(loc=12, column='backers_count', value=0)

In [21]:
test.loc[test['log_goal'] <= 1  , 'log_goal'] = 0 
test.loc[test['log_goal'] >1  & (test['log_goal']<= 3) , 'log_goal'] = 1  
test.loc[test['log_goal'] > 3 & (test['log_goal']<=5), 'log_goal'] = 2  
test.loc[test['log_goal'] > 5 , 'log_goal'] = 3  

In [22]:
test.head(1)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,duration,log_goal
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,2015-12-08 15:59:45,2015-12-08 15:59:45,2015-10-27 20:23:01,2015-10-29 15:59:45,0,5,1.0


In [23]:
my_features = ['country', 'currency', 'duration', 'goal'] #'backers_count']

In [24]:
dtrain = train[my_features]
dtest = test[my_features]

In [25]:
y = train.final_status


X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.1, random_state=0)
categorical_cols = [cname for cname in dtrain.columns if
                    dtrain[cname].nunique() < 10 and 
                    dtrain[cname].dtype == "object"]

numerical_cols = [cname for cname in dtrain.columns if 
                dtrain[cname].dtype in ['int64', 'float64']]

In [26]:
X_train.shape


(97316, 16)

In [27]:
numerical_transformer = SimpleImputer(strategy='mean')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

sc = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [28]:
Results = pd.DataFrame({'Model': [], 'Accuracy Score': []})

In [29]:
knnc = KNeighborsClassifier()

knn = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', knnc)
])

knn.fit(X_train, y_train)

predsKNN = knn.predict(X_valid)


print('Accuracy:', accuracy_score(y_valid, predsKNN))
print('CR:', classification_report(y_valid, predsKNN))
print('CM:',confusion_matrix(y_valid, predsKNN))

res = pd.DataFrame({'Model': ['KNN'],
                    'Accuracy Score': [accuracy_score(y_valid, predsKNN)]})
Results = Results.append(res)

Accuracy: 0.6297974660131324
CR:               precision    recall  f1-score   support

           0       0.71      0.79      0.75      7443
           1       0.37      0.28      0.32      3370

    accuracy                           0.63     10813
   macro avg       0.54      0.53      0.53     10813
weighted avg       0.60      0.63      0.61     10813

CM: [[5865 1578]
 [2425  945]]


In [30]:
print('Accuracy:', accuracy_score(y_valid, predsKNN))
print('CR:', classification_report(y_valid, predsKNN))
print('CM:',confusion_matrix(y_valid, predsKNN))

res = pd.DataFrame({'Model': ['KNN'],
                    'Accuracy Score': [accuracy_score(y_valid, predsKNN)]})
Results = Results.append(res)

Accuracy: 0.6297974660131324
CR:               precision    recall  f1-score   support

           0       0.71      0.79      0.75      7443
           1       0.37      0.28      0.32      3370

    accuracy                           0.63     10813
   macro avg       0.54      0.53      0.53     10813
weighted avg       0.60      0.63      0.61     10813

CM: [[5865 1578]
 [2425  945]]


In [31]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators= 200, learning_rate = 0.1, max_depth=2, use_label_encoder=False)

xgb = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model_xgb', model_xgb)
])


xgb.fit(X_train, y_train)

[10:45:09] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1635105055642/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['duration', 'goal']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['currency'])])),
                ('model_xgb',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsam...
                               gamma=0, gpu_id=-1, importance_type=No

In [32]:
with open("model_k.pkl","wb") as f:
    pkl.dump(model_xgb, f)

In [33]:
with open("model_k.pkl", 'rb') as f:
    pkl.load(f)

In [34]:
predictions = xgb.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, predictions))
print('CR:', classification_report(y_valid, predictions))
print('CM:',confusion_matrix(y_valid, predictions))

res = pd.DataFrame({'Model': ['XGB'],
                    'Accuracy Score': [accuracy_score(y_valid, predictions)]})
Results = Results.append(res)

Accuracy: 0.6916674373439379
CR:               precision    recall  f1-score   support

           0       0.70      0.97      0.81      7443
           1       0.54      0.07      0.12      3370

    accuracy                           0.69     10813
   macro avg       0.62      0.52      0.47     10813
weighted avg       0.65      0.69      0.60     10813

CM: [[7248  195]
 [3139  231]]


In [35]:
model = RandomForestClassifier(n_estimators=100, random_state=0,criterion='entropy')


clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('sc',StandardScaler()),
                      ('model', model)
])


clf.fit(X_train, y_train)


preds = clf.predict(X_valid)
print('Accuracy:', accuracy_score(y_valid, preds))
print('CR:', classification_report(y_valid, preds))
print('CM:',confusion_matrix(y_valid, preds))

res = pd.DataFrame({'Model': ['RandomForest'],
                    'Accuracy Score': [accuracy_score(y_valid, preds)]})
Results = Results.append(res)

Accuracy: 0.675020808286322
CR:               precision    recall  f1-score   support

           0       0.70      0.93      0.80      7443
           1       0.41      0.10      0.17      3370

    accuracy                           0.68     10813
   macro avg       0.56      0.52      0.48     10813
weighted avg       0.61      0.68      0.60     10813

CM: [[6949  494]
 [3020  350]]
